<a href="https://colab.research.google.com/github/Seventeee/CSC44800/blob/main/Csc_448_A3_Lun_Qu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import linear_model # for linear regression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import cross_val_score


from sklearn.preprocessing import PolynomialFeatures

<H2> Part 1 Loading the dataset </H2>

In [2]:
# read data from csv file into pandas dataframe.
# read_*() function can be use to read multiple types files, eg read_excel()
slime_size_change = pd.read_csv("https://raw.githubusercontent.com/profmcnich/example_notebook/main/science_data_large.csv")

<H3>Output the first 15 rows of the data:</H3>

In [3]:
slime_size_change.head(15)

,Temperature °C,Mols KCL,Size nm^3
0,469,647,6.244743e+05
1,403,694,5.779610e+05
2,302,975,6.196847e+05
3,779,916,1.460449e+06
4,901,18,4.325726e+04
5,545,637,7.124634e+05
6,660,519,7.006960e+05
7,143,869,2.718260e+05
8,89,461,8.919803e+04
9,294,776,4.770210e+05


<H3> Display the table information: </H3>
<H4> Check the data types of information: </H4>


In [4]:
slime_size_change.dtypes

Temperature °C      int64
Mols KCL            int64
Size nm^3         float64
dtype: object

<H4> More detail information of data, eg mean,std: </H4>

In [5]:
slime_size_change.describe()

,Temperature °C,Mols KCL,Size nm^3
count,1000.000000,1000.000000,1.000000e+03
mean,500.500000,471.530000,5.086111e+05
std,288.819436,288.482872,4.474838e+05
min,1.000000,1.000000,1.611429e+01
25%,250.750000,226.750000,1.298267e+05
50%,500.500000,459.500000,3.827182e+05
75%,750.250000,710.250000,7.603211e+05
max,1000.000000,1000.000000,1.972127e+06


<H4> 1000 $\times$ 3 Table:</H4>

In [6]:
#the total table has 1000 row and 3 colume
print(slime_size_change.shape)

(1000, 3)


<H4>Base on the data from the database, we notice that size is the result of experiment that we want to do reaserch on. And the temperature and the mols of Potassium Chloride(KCL) and will be two variable X1, X2 that impart the size of slime. We want to discover a model that can explain he growth of this new life form.  </H4>

<H2>Part 2. Splitting the dataset</H2>
<H4>In this step, we want to edit the our data series, each column in the table of pandas is consider as a series, not as same as our ragular data type (array). Thus, I gonna convert them to NumPy array, And define X(feature), Y(label).</H4>

In [7]:
type(slime_size_change['Size nm^3'])

pandas.core.series.Series

In [8]:
# Take the pandas dataset and split it into our features (X) and label (y)
X = np.array(slime_size_change.drop(['Size nm^3'], 1))
y = np.array(slime_size_change['Size nm^3'])

In [9]:
# Use sklearn to split the features and labels into a training/test set. (90% train, 10% test)
# numpy.random. RandomState() This is a pseudorandom number generator.
# we can also make random_state be a exactly number, so we can control the model we get.
rng = np.random.RandomState(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=rng)

<H2>Part 3. Perform a Linear Regression</H2>

In [10]:
# Create linear regression object
linear_clf = linear_model.LinearRegression()

In [11]:
# Train the model using the training sets
linear_clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
# Create a sample datapoint and predict the output of that sample with the trained model

# we randomly pick up a simple [469,647], which temperature is 89°C and the mols of KCL is 461
y_pred = linear_clf.predict(np.array([469,647]).reshape(1,2))
print("The output of this sample datapoint base on our linear model:", y_pred)

The output of this sample datapoint base on our linear model: [655678.05712659]


<H4>Base on this output we can not check the accuracy of my model, we want to have more evaluate base on score, such as Mean squared error, score, etc. For now we will requir to use our testing data set to judge the model. </H4>

In [13]:
# Make predictions using the testing set
y_pred = linear_clf.predict(X_test)
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 36070444496.78


<H2>!!The value is counted is too large to believe. We better to ignore this one</H2>

In [14]:
# We also want to check the score of linear regression
linear_confidence = linear_clf.score(X_test, y_test)
print('The score of the linear model:',linear_confidence)

The score of the linear model: 0.8761646752736477


<H4>From the result, we notice that the score of this model is 0.87616. We also can said that the confidence of our model is 87%. By reading, we know that score function return the coefficient of determination R^2 of the prediction. The coefficient of determination reflects what percentage of the fluctuation of y can be described by the fluctuation of x. 87%, I can only say it is a not bad score, but still not enough.</H4>

In [15]:
# The coefficients and intercepts:
print('Coefficients: \n', linear_clf.coef_)
print('Intercepts: \n', linear_clf.intercept_)

Coefficients: 
 [ 863.58108791 1006.12741921]
Intercepts: 
 -400305.9133335322


<H4> It is obvouis we have two coefficients for X1 and X2, base on $y(w,x) = w_0 + w_1x_1 + w_2x_2 + ...$ This format, we will have:
$$y = -400305.91333 + 863.58109x_1 + 1006.12741x_2$$
y is size of slime, $x_1$ is temperature, $x_2$ is mol of KCL</H4>


<H2>Part 4. Use Cross Validation</H2>

<H4>Similary to previous steps, the first thing we need to do is build the cross validation model, training the model with data of training set. But when we evalute, we are required to use function cross_val_score to see what the different with before. What the benefit the cross validation has.</H4>

In [16]:
# build our new model
# default C=1, degree=3
cross_clf = svm.SVR(kernel='linear')
cross_clf.fit(X_train, y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

<H4>By this method, we split our train set into 5 parts, and each part can be consider as a testing data separately. This method helps to reduce the total amount of data required, for this the testing set is redundant. But it also might have some problem since we analysis the result. </H4>

In [17]:
# cross_val_score will return the score for each part, the type will be array 
Cross_scores = cross_val_score(cross_clf, X_train, y_train, cv=5)
print("Score:", Cross_scores)

Score: [0.83759838 0.84277021 0.86562583 0.86006326 0.83486248]


<H4>The cv value is set to 5, thus there are 5 iterations of cross-validation are performed, and also we obtained 5 model scores. We found that the score of our model is not consistent, it fluctuates with different test domains.But in the overall, gap is small, and all 5 score are close to the answer we got from linear regression.</H4>

In [18]:
# The coefficients and intercepts:
print('Coefficients: \n', cross_clf.coef_)
print('Intercepts: \n', cross_clf.intercept_)

Coefficients: 
 [[ 867.53532073 1016.81622773]]
Intercepts: 
 [-424564.0712438]


<H>The new linear function:
    $$y = -424564.07124+ 867.53532x_1 + 1016.81623x_2$$</H>

<H2>Part 5. Using Polynomial Regression</H2>

In [19]:
poly = PolynomialFeatures(degree=2)
# transformed the features of X from [x1,x2] to [1,x1,x2,x1^2,x1x2,x2^2]
X_train = poly.fit_transform(X_train)
X_test = poly.fit_transform(X_test)

<H4> Now we can use this new X to train our linear model.</H4>

In [20]:
poly_clf = linear_model.LinearRegression()
poly_clf.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
# Create a sample datapoint and predict the output of that sample with the trained model

# we randomly pick up a simple [566 75], which temperature is 89°C and the mols of KCL is 461
# (ps: this is the fifteenth row in the table [566	75	9.185271e+04])
y_pred = poly_clf.predict( poly.fit_transform(np.array([566,75]).reshape(1,2)))
print("The output of this sample datapoint base on our linear model:", y_pred)

The output of this sample datapoint base on our linear model: [91852.71429248]


In [22]:
# check mean squared error between, predict y with the really y in the testing set.
y_pred = poly_clf.predict(X_test)
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))

Mean squared error: 0.00


In [23]:
poly_score = poly_clf.score(X_test,y_test)
print("Score of poly regression model:", poly_score)

Score of poly regression model: 1.0


<H4>We find that the both mean squared error and score show us the poly model is perfect prediction. But I don't think that the model we obtained is 100% accurate. We need to do more research to think about more possibilities.</H4>

In [24]:
# The coefficients and intercepts:
print('Coefficients: \n', poly_clf.coef_)
print('Intercepts: \n', poly_clf.intercept_)

Coefficients: 
 [ 0.00000000e+00  1.20000000e+01 -1.10383439e-07 -1.20916610e-11
  2.00000000e+00  2.85714287e-02]
Intercepts: 
 1.3900746125727892e-05


<H4>As I said, the new feature of X follow in form  $[1,x_1,x_2,x_1^2,x_1x_2,x_2^2]$. Base on that, add the coefficients to each term, we will get our the resultant equation(ignore outside of 5 significant figures): 
$$y = 0.00001 + 12x_1 +2x_1x_2 + 0.02857x_2^2$$</H4>